# Pixels model

> Observation model for encoding and decoding pixel images.

In [ ]:
#| default_exp models.pixel

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export 
import numpy as np
import torch
import torch.distributions as td
import torch.nn as nn
from fastcore.utils import *
from fastcore.all import *

In [ ]:
#| export

def conv_out(h_in, padding, kernel_size, stride):
    return int((h_in + 2. * padding - (kernel_size - 1.) - 1.) / stride + 1.)

def output_padding(h_in, conv_out, padding, kernel_size, stride):
    return h_in - (conv_out - 1) * stride + 2 * padding - (kernel_size - 1) - 1

def conv_out_shape(h_in, padding, kernel_size, stride):
    return tuple(conv_out(x, padding, kernel_size, stride) for x in h_in)

def output_padding_shape(h_in, conv_out, padding, kernel_size, stride):
    return tuple(output_padding(h_in[i], conv_out[i], padding, kernel_size, stride) for i in range(len(h_in)))

In [ ]:
#| export
class ObsEncoder(nn.Module):
    def __init__(self, input_shape, embedding_size, info):
        """
        :param input_shape: tuple containing shape of input
        :param embedding_size: Supposed length of encoded vector
        """
        super(ObsEncoder, self).__init__()
        self.shape = input_shape
        activation = info['activation']
        d = info['depth']
        k  = info['kernel']
        self.k = k
        self.d = d
        self.convolutions = nn.Sequential(
            nn.Conv2d(input_shape[0], d, k),
            activation(),
            nn.Conv2d(d, 2*d, k),
            activation(),
            nn.Conv2d(2*d, 4*d, k),
            activation(),
        )
        if embedding_size == self.embed_size:
            self.fc_1 = nn.Identity()
        else:
            self.fc_1 = nn.Linear(self.embed_size, embedding_size)


In [ ]:
#| export
@patch
def forward(self: ObsEncoder, obs):
    batch_shape = obs.shape[:-3]
    batch_shape = obs.shape[:-3]
    img_shape = obs.shape[-3:]
    embed = self.convolutions(obs.reshape(-1, *img_shape))
    embed = torch.reshape(embed, (*batch_shape, -1))
    embed = self.fc_1(embed)
    return embed


In [ ]:
#| export
@patch(as_prop=True)
def embed_size(self: ObsEncoder):
    conv1_shape = conv_out_shape(self.shape[1:], 0, self.k, 1)
    conv2_shape = conv_out_shape(conv1_shape, 0, self.k, 1)
    conv3_shape = conv_out_shape(conv2_shape, 0, self.k, 1)
    embed_size = int(4*self.d*np.prod(conv3_shape).item())
    return embed_size

In [ ]:
#| export
class ObsDecoder(nn.Module):
    def __init__(self, output_shape, embed_size, info):
        """
        :param output_shape: tuple containing shape of output obs
        :param embed_size: the size of input vector, for dreamerv2 : modelstate 
        """
        super(ObsDecoder, self).__init__()
        c, h, w = output_shape
        activation = info['activation']
        d = info['depth']
        k  = info['kernel']
        conv1_shape = conv_out_shape(output_shape[1:], 0, k, 1)
        conv2_shape = conv_out_shape(conv1_shape, 0, k, 1)
        conv3_shape = conv_out_shape(conv2_shape, 0, k, 1)
        self.conv_shape = (4*d, *conv3_shape)
        self.output_shape = output_shape
        if embed_size == np.prod(self.conv_shape).item():
            self.linear = nn.Identity()
        else:
            self.linear = nn.Linear(embed_size, np.prod(self.conv_shape).item())
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(4*d, 2*d, k, 1),
            activation(),
            nn.ConvTranspose2d(2*d, d, k, 1),
            activation(),
            nn.ConvTranspose2d(d, c, k, 1),
        )

In [ ]:
#| export
@patch
def forward(self: ObsDecoder, x):
    batch_shape = x.shape[:-1]
    embed_size = x.shape[-1]
    squeezed_size = np.prod(batch_shape).item()
    x = x.reshape(squeezed_size, embed_size)
    x = self.linear(x)
    x = torch.reshape(x, (squeezed_size, *self.conv_shape))
    x = self.decoder(x)
    mean = torch.reshape(x, (*batch_shape, *self.output_shape))
    obs_dist = td.Independent(td.Normal(mean, 1), len(self.output_shape))
    return obs_dist
 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()